### setup

In [1]:
import os
import random
import PIL
import cv2
import math
import collections
import types
import numbers
import inspect
import shutil
import dill
from timeit import default_timer as timer
# std libs


from datetime import datetime
import csv
import pandas as pd
import pickle
import glob
import sys
#from time import sleep
from distutils.dir_util import copy_tree
import zipfile
import zlib

import sklearn
import sklearn.metrics

from skimage import io
from sklearn.metrics import fbeta_score

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

In [ ]:
parser = {
    'train_path': '../data/train/',
    'train_hq_path': '../data/train_hq/',
    'train_masks_path': '../data/train_masks/',
    'train_makes_file': '../data/train_masks.csv',
    'intermediate_path': '../intermediate/',
    'train_len': 385,
    'train_skip': 91,
    'val_len': 64,
    'offset': 803,
    'batch_size': 256,
    'hidden_size': 256,
    'log_every': 10,
    'read_from_file': False,
    'train': True,
    'model_name': '',
    'test': False,
    'cuda': True,
    'seed': 20170915,
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)

args.intermediate_path = os.path.join(args.intermediate_path, str(args.seed))
if not os.path.isdir(args.intermediate_path):
    os.makedir(args.intermediate_path)

### model

In [10]:
class ConvBnRelu2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(ConvBnRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, bias=False)
        self.bn   = nn.BatchNorm2d(out_channels, eps=1e-4)
        self.relu = nn.ReLU(inplace=True)


    def forward(self, x):
        o = self.conv(x)
        if self.bn is not None:
            o = self.bn(o)
        return self.relu(o)


    def merge_bn(self):  # for faster inference
        if self.bn is None:
            return
        
        conv_weight     = self.conv.weight.data
        bn_weight       = self.bn.weight.data
        bn_bias         = self.bn.bias.data
        bn_running_mean = self.bn.running_mean
        bn_running_var  = self.bn.running_var
        bn_eps          = self.bn.eps

        N,C,H,W = conv_weight.size()
        std = torch.sqrt(bn_running_var+bn_eps)
        std_bn_weight = (bn_weight/std).repeat(C*H*W,1).t().contiguous().view(N,C,H,W)
        conv_weight_hat = std_bn_weight*conv_weight
        conv_bias_hat   = bn_bias - (bn_weight/std)*bn_running_mean
        
        self.conv = nn.Conv2d(self.conv.in_channels, self.conv.out_channels, self.conv.kernel_size,
                              padding=self.conv.padding, bias=True)
        self.conv.weight.data = conv_weight_hat
        self.conv.bias.data   = conv_bias_hat
        self.bn = None

In [9]:
class StackEncoder (nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(StackEncoder, self).__init__()
        self.encode = nn.Sequential(
            ConvBnRelu2d(in_channels, out_channels, kernel_size, padding=padding),
            ConvBnRelu2d(out_channels, out_channels, kernel_size, padding=padding),
        )

    def forward(self, x):
        e = self.encode(x)
        o = F.max_pool2d(e, kernel_size=2, stride=2)
        return e, o

In [8]:
class StackDecoder (nn.Module):
    def __init__(self, en_channels, in_channels, out_channels, kernel_size=3, padding=1):
        super(StackDecoder, self).__init__()
        self.decode = nn.Sequential(
            ConvBnRelu2d(en_channels+in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            ConvBnRelu2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            ConvBnRelu2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
        )

    def forward(self, e, x):
        N,C,H,W = e.size()
        x = F.upsample(x, size=(H,W), mode='bilinear')
        x = torch.cat([e, x], dim=1)
        return self.decode(x)

In [11]:
class UNet1024(nn.Module):
    def __init__(self, in_shape):
        super(UNet1024, self).__init__()
        C,H,W = in_shape

        # 1024
        self.down1 = StackEncoder(  C,  24, kernel_size=3)  # 512
        self.down2 = StackEncoder( 24,  64, kernel_size=3)  # 256
        self.down3 = StackEncoder( 64, 128, kernel_size=3)  # 128
        self.down4 = StackEncoder(128, 256, kernel_size=3)  # 64
        self.down5 = StackEncoder(256, 512, kernel_size=3)  # 32
        self.down6 = StackEncoder(512, 768, kernel_size=3)  # 16

        self.center = ConvBnRelu2d(768, 768, kernel_size=3, padding=1)

        # 16
        self.up6 = StackDecoder(768, 768, 512, kernel_size=3)  # 32
        self.up5 = StackDecoder(512, 512, 256, kernel_size=3)  # 64
        self.up4 = StackDecoder(256, 256, 128, kernel_size=3)  # 128
        self.up3 = StackDecoder(128, 128,  64, kernel_size=3)  # 256
        self.up2 = StackDecoder( 64,  64,  24, kernel_size=3)  # 512
        self.up1 = StackDecoder( 24,  24,  24, kernel_size=3)  # 1024
        
        self.mask = nn.Conv2d(24, 1, kernel_size=1, padding=0, bias=True)


    def forward(self, x):
        e1, o = self.down1(x)
        e2, o = self.down2(o)
        e3, o = self.down3(o)
        e4, o = self.down4(o)
        e5, o = self.down5(o)
        e6, o = self.down6(o)

        o = self.center(o)
        
        o = self.up6(e6, o)
        o = self.up5(e5, o)
        o = self.up4(e4, o)
        o = self.up3(e3, o)
        o = self.up2(e2, o)
        o = self.up1(e1, o)

        o = self.mask(o)
        o = torch.squeeze(o, dim=1)
        return o

### utils

### prepare

### train